<a href="https://colab.research.google.com/github/FranciscoOcampoPredictiva/azureml_course/blob/main/Model_Explainer_in_AzureML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - Installation an setup

In [ ]:
# Install azureml SDK package
! pip install -q azureml-sdk

     |████████████████████████████████| 135 kB 7.6 MB/s 
     |████████████████████████████████| 8.6 MB 48.9 MB/s 
     |████████████████████████████████| 3.1 MB 51.8 MB/s 
     |████████████████████████████████| 178 kB 57.2 MB/s 
     |████████████████████████████████| 1.8 MB 52.7 MB/s 
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 1.2 MB 46.8 MB/s 
     |████████████████████████████████| 605 kB 42.7 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 146 kB 50.9 MB/s 
     |████████████████████████████████| 141 kB 48.8 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 465 kB 60.9 MB/s 
     |████████████████████████████████| 212 kB 53.8 MB/s 
     |████████████████████████████████| 2.0 MB 47.3 MB/s 
     |████████████████████████████████| 83 kB 1.9 MB/s 
     |███████████████████

In [ ]:
# Install azureml interpret package
! pip install -q azureml-interpret

     |████████████████████████████████| 52 kB 19 kB/s 
     |████████████████████████████████| 3.3 MB 13.4 MB/s 
     |████████████████████████████████| 130 kB 64.2 MB/s 
     |████████████████████████████████| 356 kB 57.0 MB/s 
     |████████████████████████████████| 6.6 MB 45.0 MB/s 
     |████████████████████████████████| 34.5 MB 7.8 kB/s 


In [ ]:
# Importing the class
from azureml.core import Workspace

In [ ]:
# Access the workspace from config file and creating a workspace object
ws = Workspace.from_config(path='/content/config.json')

Performing interactive authentication. Please follow the instructions on the terminal.


Interactive authentication successfully completed.


# Step 2 - Accessing the input data

In [ ]:
input_ds = ws.datasets.get('Churn Modelling Dataset')

# Step 3 - Creating the custom environment

In [ ]:
# Importing the classes
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

# Create the environment
myenv = Environment(name="MyEnvironment")

# Create the dependencies object
myenv_dep = CondaDependencies.create(conda_packages=['scikit-learn', 'pip', 'pandas'],
                                     pip_packages=['azureml-defaults', 'azureml-sdk', 'azureml-interpret'])

myenv.python.conda_dependencies = myenv_dep

# Register environment to the workspace
myenv.register(ws)

{
    "assetId": "azureml://locations/westus/workspaces/ed1efcea-51ff-4f7c-a85d-4160d6eea5c0/environments/MyEnvironment/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220729.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "MyEnvironment",
    "pyt

# Step 4 - Create the compute Cluster

In [ ]:
cluster_name = 'azureml-sdk-cluster'

# Import the class AmlCompute
from azureml.core.compute import AmlCompute

# Provision the configuration using AmlCompute
if cluster_name not in ws.compute_targets:
  compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D11_V2',
                                                         max_nodes=2)
  cluster = AmlCompute.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)
  cluster.wait_for_completion()
else:
  cluster = ws.compute_targets[cluster_name]

Provisioning operation finished, operation "Succeeded"


# Step 5 - Script configuration for custom environment

In [ ]:
from azureml.core import ScriptRunConfig

script_config = ScriptRunConfig(source_directory=".",
                                script="training_script_for_the_explainer_job.py",
                                arguments = ['--input-data', input_ds.as_named_input('raw_data')],
                                environment=myenv,
                                compute_target=cluster)

# Step 6 - Creating and submitting the Experiment Run

In [ ]:
# Import the class Experiment
from azureml.core.experiment import Experiment
new_experiment = Experiment(workspace=ws, name='Model_Explainer_Experiment')
new_run = new_experiment.submit(config=script_config)

new_run.wait_for_completion(show_output=True)

Submitting /content directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


Streaming output truncated to the last 5000 lines.
openssl-1.1.1q       | 3.8 MB    | ########## | 100% 

ld_impl_linux-64-2.3 | 732 KB    |            |   0% 
ld_impl_linux-64-2.3 | 732 KB    | ########## | 100% 
ld_impl_linux-64-2.3 | 732 KB    | ########## | 100% 

xz-5.2.5             | 389 KB    |            |   0% 
xz-5.2.5             | 389 KB    | ########## | 100% 

pyparsing-3.0.4      | 78 KB     |            |   0% 
pyparsing-3.0.4      | 78 KB     | ########## | 100% 

zlib-1.2.12          | 130 KB    |            |   0% 
zlib-1.2.12          | 130 KB    | ########## | 100% 

bottleneck-1.3.5     | 125 KB    |            |   0% 
bottleneck-1.3.5     | 125 KB    | #2         |  13% 
bottleneck-1.3.5     | 125 KB    | ########## | 100% 

joblib-1.1.0         | 211 KB    |            |   0% 
joblib-1.1.0         | 211 KB    | ########## | 100% 

ncurses-6.3          | 1.1 MB    |            |   0% 
ncurses-6.3          | 1.1 MB    | ########## | 100% 
ncurses-6.3          | 1

{'runId': 'Model_Explainer_Experiment_1664042304_62283b58',
 'target': 'azureml-sdk-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-24T18:12:05.22857Z',
 'endTimeUtc': '2022-09-24T18:26:07.338755Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '72ecd2cf-347f-4605-91ed-cdc182184121',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'b8f67408-9beb-43f6-8320-cbca9c977620'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'raw_data', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'training_script_for_the_explainer_job.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--input-data', 'DatasetConsumptionConfig:raw_data'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'azureml-sdk-cluster',
  'dataReferences': {},
  'da